In [ ]:
"""
pyswmm example: Created by Kazi Tamaddun: 02-03-2021
This script looks at different pyswmm functionalities specific to LID Controls
"""

In [ ]:
# !conda install pyswmm
# !conda install swmm
from pyswmm import Simulation, LidControls, LidGroups

In [ ]:
# Import required packages
import os
import numpy as np

In [ ]:
# Scotts Level Brance (SLB) Model developed in EPA SWMM
inp_dir = './SLB_SWMM'
# Make sure to update the input directory

# Load EPA SWMM Model
for file in os.scandir(inp_dir):
    if file.name.endswith('.inp'):
        swmm_inp = file.path # SWMM input file 
        
        control_time_step = 86400*3 # 3 days in seconds - This will be used as the routing step
        # Setting a higher time_step just for printing convenience on Jupyter Runtime
        # The actual simulation still runs at the default value set through EPA SWMM
        
# Trying out different combinations of LID Unit_numbers and LID Unit_Areas 
        uNumbers = np.arange(1,3,1) # Two Options
        uAreas = np.arange(60000,80000,10000) # Two Options
        
        for uArea in uAreas:
            for uNumber in uNumbers:
                with Simulation(swmm_inp, reportfile = './pyswmm_Reports/Report_{}_{}.txt'.format(uArea,uNumber)) as sim:
                    # Make sure to update the output directory
                    # The report files will be in this format: 'Report_uArea_uNumber.txt'
                    
                    sim.step_advance(control_time_step)
# LID Controls Module - This lets you access or change individual LID parameters
                    bioRet = LidControls(sim)['BioRet1']
                    # I have a LID Contol named 'BioRet1'
        
                    # I am only printing two parameters here
                    print('_____________New Run!____________','\n')
                    print('Soil Thickness:',bioRet.soil.thickness,'\n',"Surface Manning's n:",bioRet.surface.roughness)
                    # The following code tells you whether immediate outflow of excess water is allowed or not
                    # You can also change the value (takes only boolean inputs)
                    print(); print(bioRet.can_overflow)

                    # Assign new parameter values
                    bioRet.soil.thickness = 700
                    bioRet.surface.roughness = 0.15 # This is Manning's n
                    print(); print('New Soil Thickness:',bioRet.soil.thickness,
                                   '\n',"New Surface Manning's n:",bioRet.surface.roughness); print()

# LID Groups Module - This lets you access the LIDs assigned to particular Subcatchment
                    lid_sub1 = LidGroups(sim)['S1']
                    # I have a subcatchment named 'S1'
                    print('Number of LID Controls:',len(lid_sub1)) # Checking how many LIDs I have in this subcatchment
                    for lid in lid_sub1:
                        print(lid) 

# LID Units Module - This lets you change LID parameters set within a subcatchment    
                        lid_sub1[0].unit_area = uArea
                        lid_sub1[0].number = uNumber
                        lid_sub1[0].from_pervious = 10

# Run the simulation
                    for step in sim:
                        print(); print('Current Time:',sim.current_time); print('LID Group Pervious Area:',lid_sub1.pervious_area);
                        print('New LID Unit Area:',lid_sub1[0].unit_area); print('New LID Number:',lid_sub1[0].number)
                        print('% From Pervious',lid_sub1[0].from_pervious); print()
                        pass

                    sim.report()
                    sim.close()